In [0]:
import requests
import json
import pandas as pd
from datetime import datetime
from pyspark.sql import SparkSession

# OpenWeather API Key
API_KEY = "f42013d0765087d965f0c96a5264f34b"

# List of Nigerian states with their coordinates
states_coords = {
    "Abia": {"lat": 5.5200, "lon": 7.4900},
    "Adamawa": {"lat": 9.3000, "lon": 12.4600},
    "Akwa Ibom": {"lat": 5.0000, "lon": 7.9000},
    "Anambra": {"lat": 6.2100, "lon": 7.0700},
    "Bauchi": {"lat": 10.3100, "lon": 9.8300},
    "Bayelsa": {"lat": 4.9220, "lon": 6.2580},
    "Benue": {"lat": 7.7361, "lon": 8.5395},
    "Borno": {"lat": 11.8460, "lon": 13.1570},
    "Cross River": {"lat": 4.9580, "lon": 8.3260},
    "Delta": {"lat": 6.2056, "lon": 6.7259},
    "Ebonyi": {"lat": 6.3220, "lon": 8.1120},
    "Edo": {"lat": 6.3320, "lon": 5.6030},
    "Ekiti": {"lat": 7.6170, "lon": 5.2170},
    "Enugu": {"lat": 6.4433, "lon": 7.5130},
    "Gombe": {"lat": 10.2892, "lon": 11.1593},
    "Imo": {"lat": 5.4833, "lon": 7.0333},
    "Jigawa": {"lat": 12.2620, "lon": 9.3280},
    "Kaduna": {"lat": 10.5200, "lon": 7.4400},
    "Kano": {"lat": 12.0022, "lon": 8.5919},
    "Katsina": {"lat": 12.9933, "lon": 7.6111},
    "Kebbi": {"lat": 12.4536, "lon": 4.2023},
    "Kogi": {"lat": 7.8036, "lon": 6.7362},
    "Kwara": {"lat": 8.4900, "lon": 4.5500},
    "Lagos": {"lat": 6.5244, "lon": 3.3792},
    "Nasarawa": {"lat": 8.2043, "lon": 8.5069},
    "Niger": {"lat": 9.6167, "lon": 6.5500},
    "Ogun": {"lat": 7.1403, "lon": 3.3500},
    "Ondo": {"lat": 7.2500, "lon": 5.1900},
    "Osun": {"lat": 7.7750, "lon": 4.5600},
    "Oyo": {"lat": 7.3775, "lon": 3.8950},
    "Plateau": {"lat": 9.9200, "lon": 9.3200},
    "Rivers": {"lat": 4.8152, "lon": 7.0498},
    "Sokoto": {"lat": 13.0700, "lon": 5.2400},
    "Taraba": {"lat": 9.3200, "lon": 11.3800},
    "Yobe": {"lat": 11.9644, "lon": 11.8444},
    "Zamfara": {"lat": 12.1642, "lon": 6.6613},
    "FCT (Abuja)": {"lat": 9.0579, "lon": 7.4951}
}

# Function to fetch weather data
def fetch_weather_data(state, lat, lon):
    url = f"http://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={API_KEY}&units=metric"
    response = requests.get(url)
    data = response.json()
    
    if response.status_code == 200:
        return {
            "state": state,
            "lat": lat,
            "lon": lon,
            "weather": data["weather"][0]["description"],
            "temperature": data["main"]["temp"],
            "humidity": data["main"]["humidity"],
            "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }
    else:
        return {
            "state": state,
            "lat": lat,
            "lon": lon,
            "weather": "N/A",
            "temperature": "N/A",
            "humidity": "N/A",
            "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }

# Fetch data for all states
weather_data_list = [fetch_weather_data(state, coords["lat"], coords["lon"]) for state, coords in states_coords.items()]

# Convert to Pandas DataFrame
weather_df = pd.DataFrame(weather_data_list)
print(weather_df.head())  # Preview the data


In [0]:
# Convert Pandas DataFrame to Spark DataFrame
spark_weather_df = spark.createDataFrame(weather_df)

# Save to Delta Table
spark_weather_df.write \
    .format("delta") \
    .mode("append") \
    .saveAsTable("nigeria_weather_data")
